# Probability

**Start downloading this file and place it in example_pcaps: https://drive.google.com/file/d/1Lr1dleCbZcQWfHoW_u6Q2uZFte17Y2Z_/view?usp=sharing. You'll need it later!**

## Introduction

Today, we're going to explore probability. The concept of probability is a powerful tool that lets us answer interesting questions about our data, and it serves as the foundation of a commonly used machine learning technique for classification We'll also be building a Naive Bayes classifier from scratch, so you'll get hands-on experience coding a machine learning classifier from scratch!

Let's start with some simple probability examples on the board. Let's see how much you can recall from lecture!

Say I have a bucket with 10 blue balls and 20 red balls. If I choose a ball at random from the bucket, what is the probability that I choose a red ball? That is, we want to calculate:

$P($red ball$)\ =\ ??$

This is equal to the fraction of red balls over the total number of balls.

$P($red ball$)\ =\ \frac{\text{# of red balls}}{\text{# of total balls}}\ =\ \frac{20}{30}\ =\ \frac{2}{3}$

Similarly, the chance of picking a blue ball is:

$P($blue ball$)\ =\ \frac{\text{# of blue balls}}{\text{# of total balls}}\ =\ \frac{10}{30}\ =\ \frac{1}{3}$

Now, let's say we want to find the probability of picking a red ball out of the bucket, **AND THEN** picking a blue ball out of the bucket. When we want to find the probability of two events both occurring, we multiply their probabilities together. The resulting probability is:

$P(\text{red ball})*P(\text{blue ball}\ |\ \text{red ball missing})$

Here, we introduce the concept of conditional probability. $P(\text{blue ball}\ |\ \text{red ball missing})$ represents the probability that a blue ball is pulled from the bucket, **given** that a red ball has already been taken out.

Are these two events independent? Does pulling a red ball affect the result of the probability of pulling a red ball followed by a blue ball? If it had no effect, the overall probability would be equivalent to:

$P(\text{red ball})*P(\text{blue ball})$

But it's not! By removing a red ball, there are now fewer overall balls to choose from, which changes the resulting probability. The full probability is therefore calculated as:

$P(\text{red ball})*P(\text{blue ball}\ |\ \text{red ball missing})\ =\ \frac{20}{30}*\frac{10}{29}\ =\ \frac{20}{87}$

Now that you've had a chance to review, let's dive into the data.

## Part 1: Probability Analysis on Network Traffic

### Probability of a TCP Packet

Let's compute the probability that a packet from our capture was a TCP packet:

$P(\text{TCP Packet})\ =\ \frac{\text{# of TCP packets}}{\text{# of total packets}}$

We'll start by loading some captured data into Python, and filtering out packets that don't have a DNS query field or a DNS response field. You'll need to fill in the blanks with the correct information. For tcp_packets, there are three options for each blank: "data", "protocol", or "TCP". Consult yesterday's lab if you need!

In [1]:
from data_collection.parse_pcap import pcap_to_pandas
from utils import *
import pandas as pd
import numpy as np
from datetime import datetime, timezone
from sklearn.preprocessing import LabelEncoder
from math import log

In [2]:
# call our helper "pcap_to_pandas" function, and pass in the argument "example_pcaps/tplink_switch.pcap"
data = pcap_to_pandas('example_pcaps/tplink_switch.pcap') 

Take a quick look at the data.  How many packets are there?

In [3]:
num_total_packets = len(data) # number of total packets
print(data.shape)

(207, 18)


In [4]:
data.head(n=5)

,datetime,dns_query,dns_resp,ip_dst,ip_dst_int,ip_src,ip_src_int,is_dns,length,mac_dst,mac_dst_int,mac_src,mac_src_int,port_dst,port_src,protocol,time,time_normed
0,2017-12-07 14:11:24.315450,None,None,None,NaN,None,NaN,False,113,50:c7:bf:09:f3:4c,88818833814348,b8:27:eb:2d:24:15,202481588839445,NaN,NaN,None,1.512677e+09,0.000000
1,2017-12-07 14:11:24.415702,None,None,None,NaN,None,NaN,False,113,50:c7:bf:09:f3:4c,88818833814348,b8:27:eb:2d:24:15,202481588839445,NaN,NaN,None,1.512677e+09,0.100252
2,2017-12-07 14:11:25.240070,None,None,None,NaN,None,NaN,False,135,b8:27:eb:2d:24:15,202481588839445,50:c7:bf:09:f3:4c,88818833814348,NaN,NaN,None,1.512677e+09,0.924620
3,2017-12-07 14:11:25.241256,None,None,None,NaN,None,NaN,False,169,50:c7:bf:09:f3:4c,88818833814348,b8:27:eb:2d:24:15,202481588839445,NaN,NaN,None,1.512677e+09,0.925806
4,2017-12-07 14:11:25.242138,None,None,None,NaN,None,NaN,False,135,b8:27:eb:2d:24:15,202481588839445,50:c7:bf:09:f3:4c,88818833814348,NaN,NaN,None,1.512677e+09,0.926688


### Basic Probability: TCP Packets

In [5]:
# packets with the protocol column equal to "TCP"
tcp_packets = data.loc[data['protocol'] == 'TCP'] 
tcp_packets.head(n=5)

,datetime,dns_query,dns_resp,ip_dst,ip_dst_int,ip_src,ip_src_int,is_dns,length,mac_dst,mac_dst_int,mac_src,mac_src_int,port_dst,port_src,protocol,time,time_normed
18,2017-12-07 14:11:31.780800,None,None,34.195.88.49,5.832274e+08,172.24.1.81,2.887254e+09,False,74,b8:27:eb:2d:24:15,202481588839445,50:c7:bf:09:f3:4c,88818833814348,50443.0,55594.0,TCP,1.512677e+09,7.465350
19,2017-12-07 14:11:31.790298,None,None,172.24.1.81,2.887254e+09,34.195.88.49,5.832274e+08,False,74,50:c7:bf:09:f3:4c,88818833814348,b8:27:eb:2d:24:15,202481588839445,55594.0,50443.0,TCP,1.512677e+09,7.474848
20,2017-12-07 14:11:31.791840,None,None,34.195.88.49,5.832274e+08,172.24.1.81,2.887254e+09,False,66,b8:27:eb:2d:24:15,202481588839445,50:c7:bf:09:f3:4c,88818833814348,50443.0,55594.0,TCP,1.512677e+09,7.476390
23,2017-12-07 14:11:32.208559,None,None,34.195.88.49,5.832274e+08,172.24.1.81,2.887254e+09,False,281,b8:27:eb:2d:24:15,202481588839445,50:c7:bf:09:f3:4c,88818833814348,50443.0,55594.0,TCP,1.512677e+09,7.893109
24,2017-12-07 14:11:32.216527,None,None,172.24.1.81,2.887254e+09,34.195.88.49,5.832274e+08,False,66,50:c7:bf:09:f3:4c,88818833814348,b8:27:eb:2d:24:15,202481588839445,55594.0,50443.0,TCP,1.512677e+09,7.901077


In [7]:
print(tcp_packets.shape)

(142, 18)


In [9]:
# len gives the number of packets in some data
# number of TCP packets
num_tcp_packets = len(tcp_packets)
print(tcp_packets.shape)
tcp_probability = num_tcp_packets / num_total_packets # probability that a packet is a TCP packet

print(tcp_probability)

(142, 18)
0.6859903381642513


### Basic Probability: UDP Packets

In [10]:
# len gives the number of packets in some data
# number of TCP packets
udp_packets = data.loc[data['protocol'] == 'UDP']
num_udp_packets = len(udp_packets) 
print(udp_packets.shape)

udp_probability = num_udp_packets / num_total_packets # probability that a packet is a UDP packet

print(udp_probability)

(32, 18)
0.15458937198067632


### Conditional Probability: DNS Packets Given Port 53

A DNS packet is a DNS query **OR** a DNS response. DNS traffic is typically sent or received on port 53. Let's compute the probability that a packet is a DNS packet, given that the source port or destination port is 53. 

We are calculating:

$P($DNS Query $\cup$ DNS Response | Source Port == 53 $\cup$ Dst Port == 53$)$

The $\cup$ means "union".

The probability can be calculated as:

$P(\text{DNS Query} \cup \text{DNS Response}\ |\ \text{Source Port == 53} \cup \text{Dst Port == 53})\ =\ \frac{\text{# of packets with a DNS query or DNS response field}}{\text{# of packets with a SRC port or DST port 53}}$

According to conditional probability, rather than dividing by the total number of packets, we divide by only the # of packets that satisfy the condition that the SRC or DST port is equal to 53.

For dns_queries and dns_responses, there are three options for each blank: "data", "dns_query", or "dns_resp". 

In [18]:
# packets with a DNS query column that isn't None
dns_queries = data.loc[data["dns_query"].notnull()] # This produces garbage.
dns_queries = dns_queries.loc[dns_queries["port_dst"] == 53]

# packets with a DNS response column that isn't Nonedns_queries = dns_queries.loc[dns_queries["port_dst"] == 53]
dns_responses = data.loc[data["dns_resp"].notnull()] # This produces garbage.
dns_responses = dns_responses.loc[dns_responses["port_src"] == 53]

dns_responses.head(n=6)

,datetime,dns_query,dns_resp,ip_dst,ip_dst_int,ip_src,ip_src_int,is_dns,length,mac_dst,mac_dst_int,mac_src,mac_src_int,port_dst,port_src,protocol,time,time_normed
17,2017-12-07 14:11:31.775682,b'devs.tplinkcloud.com.',b'devs.tplinkcloud.com.',172.24.1.81,2.887254e+09,172.24.1.1,2.887254e+09,True,533,50:c7:bf:09:f3:4c,88818833814348,b8:27:eb:2d:24:15,202481588839445,43866.0,53.0,UDP,1.512677e+09,7.460232
21,2017-12-07 14:11:31.885528,b's1a.time.edu.cn.',b's1a.time.edu.cn.',172.24.1.81,2.887254e+09,172.24.1.1,2.887254e+09,True,121,50:c7:bf:09:f3:4c,88818833814348,b8:27:eb:2d:24:15,202481588839445,32835.0,53.0,UDP,1.512677e+09,7.570078
58,2017-12-07 14:11:47.952146,b's1b.time.edu.cn.',b's1b.time.edu.cn.',172.24.1.81,2.887254e+09,172.24.1.1,2.887254e+09,True,91,50:c7:bf:09:f3:4c,88818833814348,b8:27:eb:2d:24:15,202481588839445,39900.0,53.0,UDP,1.512678e+09,23.636696
70,2017-12-07 14:12:04.028974,b'0.cn.pool.ntp.org.',b'0.cn.pool.ntp.org.',172.24.1.81,2.887254e+09,172.24.1.1,2.887254e+09,True,141,50:c7:bf:09:f3:4c,88818833814348,b8:27:eb:2d:24:15,202481588839445,55754.0,53.0,UDP,1.512678e+09,39.713524
119,2017-12-07 14:16:20.105092,b'fr.pool.ntp.org.',b'fr.pool.ntp.org.',172.24.1.81,2.887254e+09,172.24.1.1,2.887254e+09,True,513,50:c7:bf:09:f3:4c,88818833814348,b8:27:eb:2d:24:15,202481588839445,34673.0,53.0,UDP,1.512678e+09,295.789642
123,2017-12-07 14:16:20.364034,b'devs.tplinkcloud.com.',b'devs.tplinkcloud.com.',172.24.1.81,2.887254e+09,172.24.1.1,2.887254e+09,True,533,50:c7:bf:09:f3:4c,88818833814348,b8:27:eb:2d:24:15,202481588839445,59227.0,53.0,UDP,1.512678e+09,296.048584


We should expect one response for each query. Let's check that assumption.

In [17]:
print(dns_queries.shape)
print(dns_responses.shape)

(6, 18)
(6, 18)


Let's now see how many packets have either a source port of 53 or a destination port of 53.

In [15]:
src_port_53 = data.loc[data["port_src"] == 53]
dst_port_53 = data.loc[data["port_dst"] == 53]
print(src_port_53.shape)
print(dst_port_53.shape)

(6, 18)
(6, 18)


In [19]:
num_dns_queries = len(dns_queries)
num_dns_responses = len(dns_responses)

num_dns_total = num_dns_queries + num_dns_responses

# check the total
print(num_dns_total)

num_port_53 = len(src_port_53) + len(dst_port_53)

# should be the same
print(num_port_53)

# probability that a packet is a DNS packet, given that at least one port is 53
dns_probability = (num_dns_queries + num_dns_responses) / num_port_53 
print(dns_probability) # Should be 1 (100%).

12
12
1.0


You should expect an answer of 100%. If you got over 100% instead, your probability is likely overcounting some packets.

### Probability that a DNS Response is Longer than the Mean Packet Length

Now let's answer the following questions about our packets: 
What is the probability that a given DNS response has a length longer than the average length of all packets?

$P($Length > Mean Length of **All** Packets | DNS Response$)$

In [20]:
# the mean length of all packets
mean_length = data['length'].mean() 
print(mean_length)

238.58937198067633


In [22]:
# the mean length of DNS responses
dns_responses['length'].mean()

322.0

In [24]:
# number of DNS packets with a length longer than mean_length
longer_than_mean = dns_responses[dns_responses["length"] > int(mean_length)] 
num_longer = len(longer_than_mean)
print(num_longer)

prob_longer = num_longer / len(dns_responses)
print(prob_longer)

3
0.5


### Homework Exercises

(Challenge!) Find the probability that a DNS request is immediately followed by a DNS response in the packet trace. This will give us an idea of how fast DNS responses are received, relative to other network traffic.

## Part 2: Naïve Bayes Classifier

Now we're going to use the naïve Bayes algorithm to predict which task a user is most likely doing given a particular packet. While there are existing python functions for performing a naive Bayes classification, we already know everything we need to do it ourselves!

### Load the Packets and the Labels

We first need to label the data with what activity was happening at the time each packet is received.

First, download the ross.pcap file at https://drive.google.com/file/d/1Lr1dleCbZcQWfHoW_u6Q2uZFte17Y2Z_/view?usp=sharing. Place it in the example_pcaps folder.

In [25]:
# Load the data, may take a few minutes.
data = pcap_to_pandas("example_pcaps/ross.pcap")
labels = pd.read_csv('example_pcaps/ross_labels.txt', header=None, names=["time", "activity"])
labels.head(n=5)

,time,activity
0,2018-07-30 14:51:41.327734,WEB
1,2018-07-30 14:54:12.815653,AUDIO
2,2018-07-30 14:56:09.083618,VIDEO
3,2018-07-30 14:58:24.929799,WEB
4,2018-07-30 14:58:33.808876,GAMING


First, let's add a UNIX timestamp (remember, this is measured in seconds since the epoch) to the data set.

In [26]:
from dateutil import tz

def convert_to_datetime(time):
    return datetime.strptime(time, '%Y-%m-%d %H:%M:%S.%f')
    
labels['datetime'] = labels['time'].apply(convert_to_datetime)

# Force GMT -0400
tzlocal = tz.gettz('CDT')
#tzlocal = datetime.now().astimezone().tzinfo
labels['timestamp'] = labels['datetime'].apply(lambda dt: dt.replace(tzinfo=tzlocal).timestamp())

labels.head(n=50)
#print(tzlocal)

,time,activity,datetime,timestamp
0,2018-07-30 14:51:41.327734,WEB,2018-07-30 14:51:41.327734,1.532980e+09
1,2018-07-30 14:54:12.815653,AUDIO,2018-07-30 14:54:12.815653,1.532980e+09
2,2018-07-30 14:56:09.083618,VIDEO,2018-07-30 14:56:09.083618,1.532981e+09
3,2018-07-30 14:58:24.929799,WEB,2018-07-30 14:58:24.929799,1.532981e+09
4,2018-07-30 14:58:33.808876,GAMING,2018-07-30 14:58:33.808876,1.532981e+09
5,2018-07-30 14:00:20.571626,INACTIVE,2018-07-30 14:00:20.571626,1.532977e+09


### Assign an activity label to each row in the packet trace.

Next, we're going to use the activity log to label the the packet trace. 

We use a **label encoder** to assign an integer label for each activity in the dataset.

In [38]:
label_encoder = LabelEncoder()
labels['label'] = label_encoder.fit_transform(labels['activity'])

print(labels.shape)
num_labels = max(labels['label'])
print(num_labels)

labels.head(n=6)
print(type(labels))

(6, 5)
4
<class 'pandas.core.frame.DataFrame'>


In [51]:
for index, row in labels.sort_values(by=['timestamp']).iterrows():
    # bug in assigning labels of activities to the packet data
    data.loc[data['time'] >= row['timestamp'], 'label'] = row['label']
    

#data.loc[data['label' == 2]]
data[data['label'] != 2].head(n=20)
data['time'].head(n=10)

0    1.532980e+09
1    1.532980e+09
2    1.532980e+09
3    1.532980e+09
4    1.532980e+09
5    1.532980e+09
6    1.532980e+09
7    1.532980e+09
8    1.532980e+09
9    1.532980e+09
Name: time, dtype: float64

In [52]:
print(labels['activity'],labels['datetime'])
print(num_labels)
data.head(n=10)

0          WEB
1        AUDIO
2        VIDEO
3          WEB
4       GAMING
5     INACTIVE
Name: activity, dtype: object 0   2018-07-30 14:51:41.327734
1   2018-07-30 14:54:12.815653
2   2018-07-30 14:56:09.083618
3   2018-07-30 14:58:24.929799
4   2018-07-30 14:58:33.808876
5   2018-07-30 14:00:20.571626
Name: datetime, dtype: datetime64[ns]
4


,datetime,dns_query,dns_resp,ip_dst,ip_dst_int,ip_src,ip_src_int,is_dns,length,mac_dst,mac_dst_int,mac_src,mac_src_int,port_dst,port_src,protocol,time,time_normed,label
0,2018-07-30 14:51:40.670566,None,None,255.255.255.255,4.294967e+09,128.112.93.99,2.154848e+09,False,184,ff:ff:ff:ff:ff:ff,281474976710655,0c:4d:e9:b0:8e:4b,13528772677195,17500.0,17500.0,UDP,1.532980e+09,0.000000,2.0
1,2018-07-30 14:51:40.670856,None,None,128.112.93.255,2.154848e+09,128.112.93.99,2.154848e+09,False,184,ff:ff:ff:ff:ff:ff,281474976710655,0c:4d:e9:b0:8e:4b,13528772677195,17500.0,17500.0,UDP,1.532980e+09,0.000290,2.0
2,2018-07-30 14:51:41.370868,None,None,162.222.44.11,2.732469e+09,128.112.92.150,2.154847e+09,False,82,04:09:73:5f:c9:00,4438636873984,a8:60:b6:01:d0:a9,185133323899049,4282.0,56524.0,TCP,1.532980e+09,0.700302,4.0
3,2018-07-30 14:51:41.370965,None,None,162.222.44.11,2.732469e+09,128.112.92.150,2.154847e+09,False,1514,04:09:73:5f:c9:00,4438636873984,a8:60:b6:01:d0:a9,185133323899049,4282.0,56524.0,TCP,1.532980e+09,0.700399,4.0
4,2018-07-30 14:51:41.370966,None,None,162.222.44.11,2.732469e+09,128.112.92.150,2.154847e+09,False,1514,04:09:73:5f:c9:00,4438636873984,a8:60:b6:01:d0:a9,185133323899049,4282.0,56524.0,TCP,1.532980e+09,0.700400,4.0
5,2018-07-30 14:51:41.370966,None,None,162.222.44.11,2.732469e+09,128.112.92.150,2.154847e+09,False,1514,04:09:73:5f:c9:00,4438636873984,a8:60:b6:01:d0:a9,185133323899049,4282.0,56524.0,TCP,1.532980e+09,0.700400,4.0
6,2018-07-30 14:51:41.370967,None,None,162.222.44.11,2.732469e+09,128.112.92.150,2.154847e+09,False,1514,04:09:73:5f:c9:00,4438636873984,a8:60:b6:01:d0:a9,185133323899049,4282.0,56524.0,TCP,1.532980e+09,0.700401,4.0
7,2018-07-30 14:51:41.370973,None,None,162.222.44.11,2.732469e+09,128.112.92.150,2.154847e+09,False,1514,04:09:73:5f:c9:00,4438636873984,a8:60:b6:01:d0:a9,185133323899049,4282.0,56524.0,TCP,1.532980e+09,0.700407,4.0
8,2018-07-30 14:51:41.370974,None,None,162.222.44.11,2.732469e+09,128.112.92.150,2.154847e+09,False,1514,04:09:73:5f:c9:00,4438636873984,a8:60:b6:01:d0:a9,185133323899049,4282.0,56524.0,TCP,1.532980e+09,0.700408,4.0
9,2018-07-30 14:51:41.370975,None,None,162.222.44.11,2.732469e+09,128.112.92.150,2.154847e+09,False,1514,04:09:73:5f:c9:00,4438636873984,a8:60:b6:01:d0:a9,185133323899049,4282.0,56524.0,TCP,1.532980e+09,0.700409,4.0


### Create training and testing sets

Finally, we're going to take 20% of the data set and reserve it as test data.

In [53]:
msk = np.random.rand(len(data)) < 0.8
train = data[msk]
test = data[~msk]

training_size = len(train)
print(training_size)

train['label'].head(n=10)

151503


0     2.0
1     2.0
2     4.0
3     4.0
6     4.0
7     4.0
8     4.0
10    4.0
11    4.0
13    4.0
Name: label, dtype: float64

## Classification

The simplest statistic we need to compute is the probability that each label occurs:

In [54]:
label_probs = np.zeros(num_labels + 1)

for i in range(num_labels + 1):
    label_probs[i] = len(train[train['label'] == i]) / training_size
    print(i, label_probs[i])

0 0.11236741186643169
1 0.29738685042540414
2 1.3201058724909737e-05
3 0.2916113872332561
4 0.2986211494161832


Next, we are going to go through the training set and tally up what values appear in different fields and how often they appear

In [56]:
ip_table = {} # The table stores tuples of the form (label, value)
ip_list = [] # The list stores every value we've seen (uniquely).

dns_table = {}
dns_list = []

port_table = {}
port_list = []

protocol_table = {}
protocol_list = []

################
# Function: update_table
# input: table, list, label, value
# output: 
# 1. update table to increment value for corresponding (label,value) key
# 2. add value to list of unique values (activities)

def update_table(table, lst, label, value):
    if value is not None:
        table[(label, value)] = table.get((label, value), 0) + 1
    if value is not None and value not in lst:
        lst.append(value)

##################
        
        
for index, row in train.iterrows():
    # for each row in the training set:
    # - count unique source and destination IPs and ports, protocols, DNS queries, and DNS responses
    update_table(ip_table, ip_list, row['label'], row['ip_src'])
    update_table(ip_table, ip_list, row['label'], row['ip_dst'])
    update_table(port_table, port_list, row['label'], row['port_src'])
    update_table(port_table, port_list, row['label'], row['port_dst'])
    update_table(protocol_table, protocol_list, row['label'], row['protocol'])
    update_table(dns_table, dns_list, row['label'], row['dns_query'])
    update_table(dns_table, dns_list, row['label'], row['dns_resp'])

Now we use these tallies to compute the logarithm of the event probabilites. 

Typically, we prefer to work with log probabilities because many of these events have very small chances of happening. When multiplied together the resulting joint probability often end up inconveniently small for computers to work with. 

Taking logarithms will not change what we are trying to do conceptually, but improves the numerical properties of the algorithm.

In [57]:
def compute_log_probs(table, lst, smoothing):
    log_probs = {}
    
    # iterate through all of the labels
    for l in range(num_labels + 1):
        total = sum([table.get((l, val), 0) for val in lst])
        
        # lst has the unique values for the keys in the table
        for val in lst:
            if (l, val) in table:
                log_probs[(l, val)] = log(table[(l, val)] + smoothing) - log(total + smoothing * (len(lst) + 1))
        
        log_probs[(l, '<UNK>')] = log(smoothing) - log(total + smoothing * (len(lst) + 1))
    
    return log_probs

ip_log_prob = compute_log_probs(ip_table, ip_list, 1e-5)
dns_log_prob = compute_log_probs(dns_table, dns_list, 1e-5)
port_log_prob = compute_log_probs(port_table, port_list, 1e-5)
protocol_log_prob = compute_log_probs(protocol_table, protocol_list, 1e-5)

Let's look at the first 5 elements to get an idea of what we just did.

In [68]:
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

top = take(5,ip_log_prob.items())
print(top)

[((0, '128.112.93.99'), -8.824558273367144), ((0, '255.255.255.255'), -7.4895586803171526), ((0, '128.112.93.255'), -7.175901263162451), ((0, '128.112.92.150'), -0.6965061868680138), ((0, '162.222.44.11'), -1.0909642712218623)]


Each entry shows the log probability of an (activity, value) pair occurring together, for each combination of activity and value.

Finally, we are ready to create the classifier. When presented with a new row of data, we simply sum all the relevant log probabilities for each class and report the class with the highest log probability.

In [70]:
##################
# Function: return log probability for a particular value

def get_log_prob(table, val, label):
    return table.get((label, val), table[(label, '<UNK>')])


##################
# Function: Naïve Bayes Classifier

def classify(row):
    best_label = -1
    best_label_score = float('-Inf')

    # For each possible activity ...
    for l in range(num_labels + 1):
        score = log(label_probs[l])
        
        # probability of seeing this src IP, given activity (label) 'l'
        # P(IP src == i | activity = <l>)
        
        score = score + get_log_prob(ip_log_prob, row['ip_src'], l)
        
        # probability of seeing this dst IP, given activity (label) 'l'
        # P(IP src == i | activity = <l>)
        score = score + get_log_prob(ip_log_prob, row['ip_dst'], l)
        
        if row['is_dns']:
            if row['dns_query'] is not None:
                score = score + get_log_prob(dns_log_prob, row['dns_query'], l)
            
            if row['dns_resp'] is not None:
                score = score + get_log_prob(dns_log_prob, row['dns_resp'], l)
        
        score = score + get_log_prob(port_log_prob, row['port_src'], l)
        score = score + get_log_prob(port_log_prob, row['port_dst'], l)
        
        score = score + get_log_prob(protocol_log_prob, row['protocol'], l)
        
        # At the end score is a large joint probability:
        # P(IP src = i AND 
        #   IP dst = i AND ... | activity = <l>)
        
        
        # Is the score we have the largest we have so far?
        # If so, this is the most likely label
        
        if score > best_label_score:
            best_label = l
            best_label_score = score
    
    return best_label

correct = 0

############
# Important: Use the test set!

for index, row in test.iterrows():
    if classify(row) == row['label']:
        correct = correct + 1

print('Accuracy: {}'.format(correct / len(test)))

Accuracy: 0.7176041722101005
